# PROJET KAYAK

## Etape 0 : Paramétrage général du projet

### Import des librairies

In [2]:
#Import des librairies nécessaires
import numpy as np
#Import de la librairie request effectuer les requêtes HTTP
import requests
#import de la librairie Pandas
import pandas as pd
#Librairie de gestion des dates
import datetime
#Pour les graphes
import plotly.graph_objects as go
#Librairie pour le scrapping
import os 
import logging
from scrapy.crawler import CrawlerProcess

#Librairie pour Amazon services
import boto3

# Librairie sql alchemy
from sqlalchemy import create_engine, text, inspect

#Time nous permet de faire des sleep
import time

#Option pour afficher la totalité des colonnes d'un DF
pd.set_option('display.max_columns', None)

### Liste des villes à gérer 

In [16]:
#Définition de la liste des villes (35)
liste_ville = ["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]

### Paramétrage du script

In [4]:
#Définition des paramètres
#Mode de fonctionnement 
#full : le scraping va chercher toutes les informations sur l'hotel (description et latitude + longitude) (+ long)
#price : le scraping va chercher uniquement les informations du prix
#Attention le mode intervient aussi dans l'écriture des infos en base de données (si full on écrase et on réécrit)
mode = 'full'

#Récupération des datas sans faire appel aux api(via fichier local)
local_mode = False
#Empêche de lancer un scrapping
no_scrap = False

#Upload sur RDS
rds_mode = False

#clés API
#Définition d'une variable de clé d'API OPENWEATHER
API_KEY_OW = os.environ['API_KEY_OW']

#Clé API pour mapbox
mapbox_access_token = os.environ['MAP_BOX_TOKEN']

#Clé amazon : Bonne pratique : à mettre en variable d'environnement
aws_key=os.environ['AWS_ACCESS_KEY_ID']
aws_secret=os.environ['AWS_SECRET_ACCESS_KEY']
bucket_name = 'jedha-kayak-q25sd4qs8d4q2'

#Param pour RDS
p_DBName="kayak_project"
p_DBInstanceIdentifier="kaykainstance"
p_MasterUsername="jedhakayakroot"
p_MasterUserPassword="lkjds4f4sd25*sd4f354s35df4"


#Date du séjour 
#info: comme l'api météo ne donne que sur 7 jours de prévision les dates du voyages doivent être entre J et J+7
date_sejour = '2023-05-06'
duree_sejour = 4

In [14]:
#Calcul Parametre date de fin du séjour
jour = datetime.date(int(date_sejour[0:4]),int(date_sejour[5:7]),int(date_sejour[8:10]))
duree = datetime.timedelta(days = duree_sejour)
date_fin_sejour = jour + duree
date_fin_sejour = date_fin_sejour.isoformat()
print('La date de fin de séjour est : '+date_fin_sejour)

La date de fin de séjour est : 2023-05-10


In [17]:
#Affichage des infos 
print('Résumé des paramètres')
print('Nombre de villes : '+str(len(liste_ville)))
print('Date arrivée : '+date_sejour)
print('Date départ : '+date_fin_sejour)
print('Nombres de jours : ' + str(duree_sejour))

Résumé des paramètres
Nombre de villes : 35
Date arrivée : 2023-05-06
Date départ : 2023-05-10
Nombres de jours : 4


## Etape 1 : Récupération des informations de l'API nominatim

In [8]:
#Boucle sur la liste des villes pour récupérer les coordonées latitude et longitude : en sortie un dictionnaire
#Appel à l'API nominatim afin de récupérer la latitude et la longitude
#Paramètres : 
#countrycodes=FR : On filtre ici directement sur la FRANCE(FR) car toutes les villes du scope sont françaises
#limit=1 : Limitation des résultats à 1, en effet, tous les résultats approchants ou moins "intéressants" n'ont pas d'interet ici
#format=json : format de retour demandé à l'API, le json c'est le plus classique
#city= XXXX : on indique ici la ville à récupérer
#TODO possible : Faire des appels asynchrones pour gagner du temps

if not local_mode:
    #initialisation du dictionnaire final qui comprendra une ville, lattitude et longitude
    dict_ville = dict()

    #Boucle sur les villes
    for ville in liste_ville:
        try:
        
            #Requete HTTP, on remplace les eventuels espaces par %20
            response = requests.get("https://nominatim.openstreetmap.org/search?countrycodes=FR&limit=1&format=json&city="+ville.replace(' ','%20'))
            
            #L'API renvoie du vide si rien n'est trouvé mais avec un code 200 (OK)
            if len(response.json()) == 0:
            
                #Si la reponse est vide pour la ville, recherche avec paramètre q= qui permet d'effectuer une recherche large, pas forcement sur une ville
                response = requests.get("https://nominatim.openstreetmap.org/search?countrycodes=FR&limit=1&format=json&q="+ville.replace(' ','%20'))
            
            #Ajout des informations récupérées dans le dictionnaire
            dict_ville.update({ville:{'lat':response.json()[0]['lat'] ,'lon':response.json()[0]['lon']}})
        
        #gestion d'erreur minimaliste
        except:
            print('un problème a eu lieu sur la récupération des informations de la ville',ville)

        
    print(dict_ville)


{'Mont Saint Michel': {'lat': '48.6359541', 'lon': '-1.511459954959514'}, 'St Malo': {'lat': '48.649518', 'lon': '-2.0260409'}, 'Bayeux': {'lat': '49.2764624', 'lon': '-0.7024738'}, 'Le Havre': {'lat': '49.4938975', 'lon': '0.1079732'}, 'Rouen': {'lat': '49.4404591', 'lon': '1.0939658'}, 'Paris': {'lat': '48.8534951', 'lon': '2.3483915'}, 'Amiens': {'lat': '49.8941708', 'lon': '2.2956951'}, 'Lille': {'lat': '50.6365654', 'lon': '3.0635282'}, 'Strasbourg': {'lat': '48.584614', 'lon': '7.7507127'}, 'Chateau du Haut Koenigsbourg': {'lat': '48.2495226', 'lon': '7.3454923'}, 'Colmar': {'lat': '48.0777517', 'lon': '7.3579641'}, 'Eguisheim': {'lat': '48.0447968', 'lon': '7.3079618'}, 'Besancon': {'lat': '47.2380222', 'lon': '6.0243622'}, 'Dijon': {'lat': '47.3215806', 'lon': '5.0414701'}, 'Annecy': {'lat': '45.8992348', 'lon': '6.1288847'}, 'Grenoble': {'lat': '45.1875602', 'lon': '5.7357819'}, 'Lyon': {'lat': '45.7578137', 'lon': '4.8320114'}, 'Gorges du Verdon': {'lat': '43.7496562', 'lon':

In [9]:
#Création d'un DF à partir du dictionnaire = en sortie le Dataframe ville_df
if not local_mode:
    #Le dictionnaire créé nous permet de créer le dataframe final, orient='index' permet d'indiqué que chaque element du dictionnaire est une ligne 
    ville_df = pd.DataFrame.from_dict(dict_ville,orient='index')
    #Par défaut le DF retourné contient la clé en index, retour a un index numérique
    ville_df.reset_index(inplace=True)
    #Clean du nom des colonnes
    ville_df.rename(columns={'index':'Ville','lat':'Latitude','lon':'Longitude'},inplace=True)
    #Affichage
    ville_df.head()

## Etape 2 : Récupération des informations de l'API OpenWeather

In [10]:
#Appel à l'api openweather : En sortie le dataframe df_result avec les données sur tous les jours de l'API + df_avg représentant les valeurs moyennes pour la période du séjour
if not local_mode:
    #Liste de dataframe finale (1 par ville )
    df_list = []
    #Boucle sur le dataframe de l'étape précédente
    for i in ville_df.index:
    #for i in range(0,3):
        #Récupération des variables utiles pour la requêtes météo 
        ville = ville_df.iloc[i,0]
        lat = ville_df.iloc[i,1]
        lon = ville_df.iloc[i,2]

        #appel API paramètres :
        #units=metric : Type d'unité (système métric)
        #lang=fr : Récupération des textes au format français
        #exclude=current,minutely,hourly,alerts : Nous cherchons des données journalières, donc on exclu les données actuelles, à la minute, à l'heure et les alertes
        #appid : Clé API
        response = requests.get("https://api.openweathermap.org/data/3.0/onecall?units=metric&lang=fr&lat="+str(lat)+"&lon="+str(lon)+"&exclude=current,minutely,hourly,alerts&appid="+API_KEY_OW)
        resp_j = response.json()  
        #Conversion d'une partie du JSON au format dataframe (la partie daily contient le forecast des 8 prochains jours)
        #La fonction json_normalize permet la mise à plat du json, utile pour les champs de daily qui contiennent un dictionnaire ou une liste
        df_t = pd.json_normalize(resp_j['daily'])
        # Ajout d'informations au Dataframe 
        df_t.insert(0, 'Timezone_offset',resp_j['timezone_offset'], allow_duplicates=False)
        df_t.insert(0, 'Timezone',resp_j['timezone'], allow_duplicates=False)
        df_t.insert(0, 'Latitude_return',resp_j['lat'], allow_duplicates=False)
        df_t.insert(0, 'Longitude_return',resp_j['lon'], allow_duplicates=False)
        df_t.insert(0, 'Latitude_asked',lat, allow_duplicates=False)
        df_t.insert(0, 'Longitude_asked',lon, allow_duplicates=False)
        df_t.insert(0, 'Ville',ville, allow_duplicates=False)
        df_t.insert(0, 'Ville_Id',i, allow_duplicates=False)
        #Ajout du dataframe Temporaire à la liste 
        df_list.append(df_t)
    
    #Concaténation du résultat en un seul dataframe avec son propre index
    df_result = pd.concat(df_list,ignore_index=True)
    #Transformation des colonnes time Unix en datetime
    df_result.insert(8, 'Sunrise',pd.to_datetime(df_result['sunrise'],unit='s'), allow_duplicates=False)
    df_result.insert(8, 'Sunset',pd.to_datetime(df_result['sunset'],unit='s'), allow_duplicates=False)
    df_result.insert(8, 'Moonrise',pd.to_datetime(df_result['moonrise'],unit='s'), allow_duplicates=False)
    df_result.insert(8, 'Moonset',pd.to_datetime(df_result['moonset'],unit='s'), allow_duplicates=False)
    df_result.insert(8, 'Date',pd.to_datetime(df_result['dt'],unit='s'), allow_duplicates=False)
    df_result.drop(columns=['sunrise', 'sunset','moonrise','moonset','dt'],inplace=True)
    #Désencapsulation de la colonne weather
    df_result.insert(20,'weather.icon',df_result['weather'][0][0]['icon'], allow_duplicates=False)
    df_result.insert(20,'weather.description',df_result['weather'][0][0]['description'], allow_duplicates=False)
    df_result.insert(20,'weather.main',df_result['weather'][0][0]['main'], allow_duplicates=False)
    df_result.insert(20,'weather.id',df_result['weather'][0][0]['id'], allow_duplicates=False)
    df_result.drop(columns=['weather'],inplace=True)


    #TODO possible rajouter le mode des valeurs de temps (la valeur la plus représenter dans la meteo (nuageux ensoleillé etc...))
    #Calcul des aggrégations des différents indicateurs
    df_avg =  df_result.groupby(['Ville_Id','Ville','Latitude_return','Longitude_return']).agg(
                avg_humidity = pd.NamedAgg(column='humidity', aggfunc="mean"),
                avg_wind_speed = pd.NamedAgg(column='wind_speed', aggfunc="mean"),
                avg_clouds = pd.NamedAgg(column='clouds', aggfunc="mean"),
                avg_pop = pd.NamedAgg(column='pop', aggfunc="mean"),
                sum_rain = pd.NamedAgg(column='rain', aggfunc="sum"),
                avg_feel_temp = pd.NamedAgg(column='feels_like.day', aggfunc="mean"),
                ).reset_index()
    
else:
    df_result=pd.read_csv('./testing_data/df_result.csv')
    df_avg=pd.read_csv('./testing_data/df_avg.csv')

In [11]:
#Sauvegarde au format CSV en local 
#Si on est en local mode, on écrit les données directement dans le dossier de retour de s3 car on ne fera pas appel a s3
if local_mode:
    df_result.to_csv('./data_from_s3/df_result.csv')
    df_avg.to_csv('./data_from_s3/df_avg.csv')
else:
    df_result.to_csv('./data/df_result.csv')
    df_avg.to_csv('./data/df_avg.csv')

## Etape 3 : Scrapping des données sur BOOKING.COM

In [12]:
#Import du spyder
from scrap_booking import BookingSpider 

#Si le paramètre général est à False on lance le scrapping sinon non
if not no_scrap:

    # Name of the file where the results will be saved
    filename = "booking_scrap.json"

    # If file already exists, delete it before crawling (because Scrapy will 
    # concatenate the last and new results otherwise)
    if filename in os.listdir('data/'):
            os.remove('data/' + filename)

    # Declare a new CrawlerProcess with some settings
    ## USER_AGENT => Simulates a browser on an OS
    ## LOG_LEVEL => Minimal Level of Log 
    ## FEEDS => Where the file will be stored 
    ## More info on built-in settings => https://docs.scrapy.org/en/latest/topics/settings.html?highlight=settings#settings
    process = CrawlerProcess(settings = {
        'USER_AGENT': 'Chrome/97.0',
        'LOG_LEVEL': logging.INFO,
        "FEEDS": {
            'data/' + filename : {"format": "json"},
        
        },
        'FEED_EXPORT_ENCODING' : 'utf-8'
    })


    # Start the crawling using the spider you defined above, en mode full on va chercher les informations sur une autre page 
    # ce qui augmente la durée de traitement par hotel
    process.crawl(BookingSpider,list_ville=liste_ville,date_in=date_sejour,date_out=date_fin_sejour,refresh_mode = mode)
    process.start()


2023-05-04 19:02:34 [scrapy.utils.log] INFO: Scrapy 2.8.0 started (bot: scrapybot)
2023-05-04 19:02:34 [scrapy.utils.log] INFO: Versions: lxml 4.9.2.0, libxml2 2.9.14, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.1, Twisted 22.10.0, Python 3.11.3 (main, May  4 2023, 01:51:29) [GCC 10.2.1 20210110], pyOpenSSL 23.1.1 (OpenSSL 3.1.0 14 Mar 2023), cryptography 40.0.2, Platform Linux-5.15.49-linuxkit-aarch64-with-glibc2.31
2023-05-04 19:02:34 [scrapy.crawler] INFO: Overridden settings:
{'DOWNLOAD_DELAY': 2,
 'FEED_EXPORT_ENCODING': 'utf-8',
 'LOG_LEVEL': 20,
 'USER_AGENT': 'Chrome/97.0'}
2023-05-04 19:02:34 [py.warnings] WARNING: /usr/local/lib/python3.11/site-packages/scrapy/utils/request.py:232: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward 

### Analyse de la log 

In [ ]:
#Analayse de la log : récupération du nombre de pages identifiées par ville dans la log (merci excel)
2023-02-17 19:58:15 [root] INFO: Ville : Mont Saint Michel Le nombre de page à scrapper : 1
2023-02-17 19:58:18 [root] INFO: Ville : St Malo Le nombre de page à scrapper : 4
2023-02-17 19:58:21 [root] INFO: Ville : Bayeux Le nombre de page à scrapper : 3
2023-02-17 19:58:24 [root] INFO: Ville : Le Havre Le nombre de page à scrapper : 3
2023-02-17 19:58:27 [root] INFO: Ville : Rouen Le nombre de page à scrapper : 5
2023-02-17 19:58:29 [root] INFO: Ville : Paris Le nombre de page à scrapper : 1
2023-02-17 19:58:31 [root] INFO: Ville : Amiens Le nombre de page à scrapper : 5
2023-02-17 19:58:34 [root] INFO: Ville : Lille Le nombre de page à scrapper : 4
2023-02-17 19:58:35 [root] INFO: Ville : Strasbourg Le nombre de page à scrapper : 5
2023-02-17 19:58:38 [root] INFO: Ville : Chateau du Haut Koenigsbourg Le nombre de page à scrapper : 5
2023-02-17 19:58:40 [root] INFO: Ville : Colmar Le nombre de page à scrapper : 5
2023-02-17 19:58:43 [root] INFO: Ville : Eguisheim Le nombre de page à scrapper : 1
2023-02-17 19:58:45 [root] INFO: Ville : Besancon Le nombre de page à scrapper : 2
2023-02-17 19:58:47 [root] INFO: Ville : Dijon Le nombre de page à scrapper : 3
2023-02-17 19:58:49 [root] INFO: Ville : Annecy Le nombre de page à scrapper : 5
2023-02-17 19:58:51 [root] INFO: Ville : Grenoble Le nombre de page à scrapper : 2
2023-02-17 19:58:54 [root] INFO: Ville : Lyon Le nombre de page à scrapper : 5
2023-02-17 21:49:18 [root] INFO: Ville : Gorges du Verdon Le nombre de page à scrapper : 4
2023-02-17 21:49:21 [root] INFO: Ville : Bormes les Mimosas Le nombre de page à scrapper : 1
2023-02-17 21:49:24 [root] INFO: Ville : Cassis Le nombre de page à scrapper : 4
2023-02-17 22:52:16 [root] INFO: Ville : Marseille Le nombre de page à scrapper : 5
2023-02-17 22:52:19 [root] INFO: Ville : Aix en Provence Le nombre de page à scrapper : 4
2023-02-17 22:52:21 [root] INFO: Ville : Avignon Le nombre de page à scrapper : 5
2023-02-17 22:52:22 [root] INFO: Ville : Uzes Le nombre de page à scrapper : 2
2023-02-17 22:52:24 [root] INFO: Ville : Nimes Le nombre de page à scrapper : 4
2023-02-17 22:52:27 [root] INFO: Ville : Aigues Mortes Le nombre de page à scrapper : 2
2023-02-17 22:52:29 [root] INFO: Ville : Saintes Maries de la mer Le nombre de page à scrapper : 2
2023-02-17 22:52:33 [root] INFO: Ville : Collioure Le nombre de page à scrapper : 2
2023-02-17 22:52:34 [root] INFO: Ville : Carcassonne Le nombre de page à scrapper : 5
2023-02-17 22:52:37 [root] INFO: Ville : Ariege Le nombre de page à scrapper : 4
2023-02-17 22:52:39 [root] INFO: Ville : Toulouse Le nombre de page à scrapper : 5
2023-02-17 22:52:41 [root] INFO: Ville : Montauban Le nombre de page à scrapper : 2
2023-02-17 22:52:42 [root] INFO: Ville : Biarritz Le nombre de page à scrapper : 5
2023-02-17 22:52:45 [root] INFO: Ville : Bayonne Le nombre de page à scrapper : 1
2023-02-17 22:52:47 [root] INFO: Ville : La Rochelle Le nombre de page à scrapper : 3

#Info : le nombre de page de ce tir a été limité à 5
#En realisant la somme des chiffres, sachant que la dernière page peut récupérer entre 1 et 25 résultats : 
#On devrait avoir entre 2100 et 2975 lignes dans le fichiers.
#Le fichier contient 2444 lignes. Ce qui parrait raisonnablement acceptable. 

#Voyons maintenant le temps d'éxecution : 425minutes soit 7heures.... 
#Un délais de 2 secondes entre chaque requête a été instauré 
    #custom_settings = {
    #    'DOWNLOAD_DELAY': 2, # 10 seconds of delay
    #    'RANDOMIZE_DOWNLOAD_DELAY': True, # By default, when you set DOWNLOAD_DELAY = 2 for example, Scrapy will introduce random delays of between: 
                                          # Lower Limit: 0.5 * DOWNLOAD_DELAY
                                          # Upper Limit: 1.5 * DOWNLOAD_DELAY
    #    'HTTPCACHE_ENABLED': False
    #}
#Voici ce que montre la log: 
#2023-02-18 03:03:14 [scrapy.extensions.logstats] INFO: Crawled 2597 pages (at 2 pages/min), scraped 2443 items (at 2 items/min)
#2597 pages appelées - 1 (la page de départ), comme le temps est randomizé entre 0,5*2sec et 1,5*2 sec on a un temps "d'attente" situé entre : 2596 sec et 7788sec soit 45 minutes et 2h15
#on peut dire environ 1h30 d'attente
#soit 5h de scrapping 


![alternative text](tab.jpg)

### Nettoyage des données Scrappées

In [13]:
#Import du fichier scrappé en DF
if no_scrap:
    #Utilisation d'un fichier de test local
    df_hotel = pd.read_json("./testing_data/booking_scrap.json")
else:
    df_hotel = pd.read_json("./data/booking_scrap.json")

In [14]:
#Affichage pour une ville
df_hotel[df_hotel['ville']=='Aix en Provence'].head()

,ville,nom,url,prix,note,distance_center,latitude,longitude,description
1224,Aix en Provence,Provencal villa with pool and terrace in Aix-e...,https://www.booking.com/hotel/fr/provencal-vil...,€ 1 704,None,"5,3 km du centre",43.480496,5.428179,"Cette villa comprend 3 chambres, un salon, une..."
1458,Aix en Provence,Un Cocon au pays de Cézanne,https://www.booking.com/hotel/fr/appartement-m...,€ 964,None,"7,8 km du centre",43.574879,5.375224,"Cet appartement comprend une terrasse, une cha..."
1459,Aix en Provence,T2 tout confort climatisé de Lilou,https://www.booking.com/hotel/fr/t2-tout-confo...,€ 581,None,1 km du centre,43.523789,5.433065,"Cet appartement comprend une chambre, une cuis..."
1460,Aix en Provence,Studio centre historique climatisation,https://www.booking.com/hotel/fr/studio-centre...,€ 413,None,"0,6 km du centre",43.530506,5.450253,Cet appartement climatisé comprend une chambre...
1461,Aix en Provence,Hôtel le Prieuré,https://www.booking.com/hotel/fr/le-prieure-ai...,€ 326,"5,4","2,1 km du centre",43.542624,5.459127,Les chambres sont équipées d'un téléphone et d...


In [15]:
#Nettoyage du prix 
df_hotel['prix'] = df_hotel['prix'].apply(lambda x : float(x.replace('€','').replace(' ','')))

In [16]:
#Nettoyage distance
df_hotel['distance_center'] = df_hotel['distance_center'].str.extract(r'(\d+(?:[.,]*\d*).(?:m|km))').loc[:,0].apply(lambda x : float(str(x).replace(',','.').split('\u00a0')[0]) if str(x).find('km') else float(str(x).replace(',','.').split('\u00a0')[0])/1000 if str(x).find('m') else None)

In [17]:
#Nettoyage des notes 
df_hotel['note'] = df_hotel['note'].apply(lambda x : float(x.replace(',','.')) if x!= None else None)

In [18]:
#Affichage pour une ville
df_hotel[df_hotel['ville']=='Aix en Provence'].head()

,ville,nom,url,prix,note,distance_center,latitude,longitude,description
1224,Aix en Provence,Provencal villa with pool and terrace in Aix-e...,https://www.booking.com/hotel/fr/provencal-vil...,1704.0,NaN,5.3,43.480496,5.428179,"Cette villa comprend 3 chambres, un salon, une..."
1458,Aix en Provence,Un Cocon au pays de Cézanne,https://www.booking.com/hotel/fr/appartement-m...,964.0,NaN,7.8,43.574879,5.375224,"Cet appartement comprend une terrasse, une cha..."
1459,Aix en Provence,T2 tout confort climatisé de Lilou,https://www.booking.com/hotel/fr/t2-tout-confo...,581.0,NaN,1.0,43.523789,5.433065,"Cet appartement comprend une chambre, une cuis..."
1460,Aix en Provence,Studio centre historique climatisation,https://www.booking.com/hotel/fr/studio-centre...,413.0,NaN,0.6,43.530506,5.450253,Cet appartement climatisé comprend une chambre...
1461,Aix en Provence,Hôtel le Prieuré,https://www.booking.com/hotel/fr/le-prieure-ai...,326.0,5.4,2.1,43.542624,5.459127,Les chambres sont équipées d'un téléphone et d...


In [19]:
#Sauvegarde au format CSV en local
#Si on est en local mode, on écrit les données directement dans le dossier de retour de s3 car on ne fera pas appel a s3
if local_mode:
    df_hotel.to_csv('./data_from_s3/df_hotel.csv')
else:
    df_hotel.to_csv('./data/df_hotel.csv')


## Etape 4 : Envoi des données sur Amazon S3

In [20]:
#Connexion et vérification du bucket
#Init de la session 
if not local_mode:
    session = boto3.Session(aws_access_key_id=aws_key, 
                            aws_secret_access_key=aws_secret)
    #Connexion au service S3
    s3 = session.resource("s3")

    b_exist = False
    # check existence bucket
    for bucket_i in s3.buckets.all():
        if bucket_i.name == bucket_name:
            b_exist = True

    #Si le bucket n'existe pas on le créer
    if not b_exist:
        bucket = s3.create_bucket(Bucket=bucket_name)

    #Upload du fichier issu du scrapping + fichier météo
    s3.Bucket(bucket_name).upload_file('./data/df_hotel.csv', 'df_hotel.csv')
    s3.Bucket(bucket_name).upload_file('./data/df_avg.csv', 'df_avg.csv')
    s3.Bucket(bucket_name).upload_file('./data/df_result.csv', 'df_result.csv')
    

In [21]:
#Récupération des fichiers sur s3 dans le dossier data_from_s3 (si on est en local les fichiers ont été créés directement dedans)
if not local_mode:
        s3.Bucket(bucket_name).download_file('df_hotel.csv', './data_from_s3/df_hotel.csv')
        s3.Bucket(bucket_name).download_file('df_avg.csv', './data_from_s3/df_avg.csv')    
        s3.Bucket(bucket_name).download_file('df_result.csv', './data_from_s3/df_result.csv')

## Etape 5 : Envoi des données sur Amazon RDS

In [9]:
#Lecture des fichiers CSV redescendu de S3    
df_avg = pd.read_csv('./data_from_s3/df_avg.csv')
df_result = pd.read_csv('./data_from_s3/df_result.csv')
df_hotel = pd.read_csv('./data_from_s3/df_hotel.csv')

In [23]:
if rds_mode:
    #Connexion à Amazon RDS
    conn = boto3.client('rds', aws_access_key_id=aws_key,
                        aws_secret_access_key=aws_secret,
                        region_name='eu-west-3')

    #Vérification si la base existe 
    v_exist = False
    response = conn.describe_db_instances()
    for resp in response['DBInstances']:
        if resp['DBName'] == p_DBName:
            v_address = resp['Endpoint']['Address']
            v_port = resp['Endpoint']['Port']
            v_exist=True

    #Création de la base 
    if not v_exist:
        response = conn.create_db_instance(
                AllocatedStorage=10,
                DBName=p_DBName,
                DBInstanceIdentifier=p_DBInstanceIdentifier,
                DBInstanceClass="db.t3.micro",
                Engine="postgres",
                MasterUsername=p_MasterUsername,
                MasterUserPassword=p_MasterUserPassword
            )

        while not v_exist:
            #Attente de la création de la base
            time.sleep(60)
            response = conn.describe_db_instances()
            for resp in response['DBInstances']:
                if resp['DBName'] == p_DBName and resp['DBInstanceStatus'] == 'available':
                    v_address = resp['Endpoint']['Address']
                    v_port = resp['Endpoint']['Port']
                    v_exist=True

In [24]:
if rds_mode:
    #Connexion via sql alchemy
    DBHOST = v_address
    DBUSER = p_MasterUsername
    DBPASS = p_MasterUserPassword
    DBNAME = p_DBName
    PORT = v_port

    engine = create_engine(f"postgresql+psycopg2://{DBUSER}:{DBPASS}@{DBHOST}/{DBNAME}", echo=True)

In [25]:
#Préparation du dataframe pour RDS
df_table_hotel = df_hotel[['ville','nom','url','distance_center','latitude','longitude','description']]

In [26]:
df_table_price_hotel = df_hotel[['ville','nom','url','prix','note']]
df_table_price_hotel['date_arrivee'] = date_sejour
df_table_price_hotel['date_depart'] = date_fin_sejour

/tmp/ipykernel_49646/4290110585.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table_price_hotel['date_arrivee'] = date_sejour
/tmp/ipykernel_49646/4290110585.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table_price_hotel['date_depart'] = date_fin_sejour


In [27]:
df_table_price_hotel

,ville,nom,url,prix,note,date_arrivee,date_depart
0,Mont Saint Michel,"Mobil-Home Jullouville, 3 pièces, 4 personnes ...",https://www.booking.com/hotel/fr/maison-jullou...,594.0,9.0,2023-05-06,2023-05-10
1,St Malo,Appartement à louer pour la route du rhum 2 ch...,https://www.booking.com/hotel/fr/appartement-a...,1835.0,NaN,2023-05-06,2023-05-10
2,Bayeux,Mon Saint-Patrice by Melrose,https://www.booking.com/hotel/fr/mon-saint-pat...,856.0,NaN,2023-05-06,2023-05-10
3,Le Havre,Apart hôtel Le Havre,https://www.booking.com/hotel/fr/apart-le-havr...,396.0,NaN,2023-05-06,2023-05-10
4,Rouen,Once upon a time in Rouen,https://www.booking.com/hotel/fr/once-upon-a-t...,219.0,NaN,2023-05-06,2023-05-10
...,...,...,...,...,...,...,...
2400,Cassis,sous un olivier,https://www.booking.com/hotel/fr/sous-un-olivi...,651.0,8.5,2023-05-06,2023-05-10
2401,Cassis,"Cassis, le Grand Bleu, triplex vue mer, port p...",https://www.booking.com/hotel/fr/le-grand-bleu...,1251.0,9.0,2023-05-06,2023-05-10
2402,Cassis,LOU CIGALOU,https://www.booking.com/hotel/fr/lou-cigalou-c...,1049.0,9.0,2023-05-06,2023-05-10
2403,Cassis,La forlane,https://www.booking.com/hotel/fr/la-forlane-ca...,943.0,9.1,2023-05-06,2023-05-10


In [28]:
df_result.columns

Index(['Unnamed: 0', 'Ville_Id', 'Ville', 'Longitude_asked', 'Latitude_asked',
       'Longitude_return', 'Latitude_return', 'Timezone', 'Timezone_offset',
       'Date', 'Moonset', 'Moonrise', 'Sunset', 'Sunrise', 'moon_phase',
       'pressure', 'humidity', 'dew_point', 'wind_speed', 'wind_deg',
       'wind_gust', 'weather.id', 'weather.main', 'weather.description',
       'weather.icon', 'clouds', 'pop', 'rain', 'uvi', 'temp.day', 'temp.min',
       'temp.max', 'temp.night', 'temp.eve', 'temp.morn', 'feels_like.day',
       'feels_like.night', 'feels_like.eve', 'feels_like.morn'],
      dtype='object')

In [29]:
if rds_mode:
	#Définition des requetes sql de MAJ or INS des données
	updt_hotel = "update booking_hotel_price tb \
	set prix = tmp.prix, note=tmp.note \
	from booking_hotel_price_tmp tmp \
	where tb.ville = tmp.ville \
	and tb.nom = tmp.nom \
	and tb.url = tmp.url \
	and tb.date_arrivee = tmp.date_arrivee \
	and tb.date_depart = tmp.date_depart"


	ins_hotel = "insert into booking_hotel_price \
	select * \
	from booking_hotel_price_tmp tmp \
	where not exists (select 1 from booking_hotel_price tb \
					where tb.ville = tmp.ville \
					and tb.nom = tmp.nom \
					and tb.url = tmp.url \
					and tb.date_arrivee = tmp.date_arrivee \
					and tb.date_depart = tmp.date_depart)"

	updt_meteo = 'update meteo tb \
	set \
	"Longitude_asked" = tmp."Longitude_asked", \
	"Latitude_asked" = tmp."Latitude_asked", \
	"Longitude_return" = tmp."Longitude_return", \
	"Latitude_return" = tmp."Latitude_return", \
	"Timezone" = tmp."Timezone", \
	"Timezone_offset" = tmp."Timezone_offset", \
	"Moonset" = tmp."Moonset", \
	"Moonrise" = tmp."Moonrise", \
	"Sunset" = tmp."Sunset", \
	"Sunrise" = tmp."Sunrise", \
	"moon_phase" = tmp."moon_phase", \
	"pressure" = tmp."pressure", \
	"humidity" = tmp."humidity", \
	"dew_point" = tmp."dew_point", \
	"wind_speed" = tmp."wind_speed", \
	"wind_deg" = tmp."wind_deg", \
	"wind_gust" = tmp."wind_gust", \
	"weather.id" = tmp."weather.id", \
	"weather.main" = tmp."weather.main", \
	"weather.description" = tmp."weather.description", \
	"weather.icon" = tmp."weather.icon", \
	"clouds" = tmp."clouds", \
	"pop" = tmp."pop", \
	"uvi" = tmp."uvi", \
	"temp.day" = tmp."temp.day", \
	"temp.min" = tmp."temp.min", \
	"temp.max" = tmp."temp.max", \
	"temp.night" = tmp."temp.night", \
	"temp.eve" = tmp."temp.eve", \
	"temp.morn" = tmp."temp.morn", \
	"feels_like.day" = tmp."feels_like.day", \
	"feels_like.night" = tmp."feels_like.night", \
	"feels_like.eve" = tmp."feels_like.eve", \
	"feels_like.morn" = tmp."feels_like.morn", \
	"rain" = tmp."rain", \
	"snow" = tmp."snow" \
	from meteo_tmp tmp  \
	where tb."Ville" = tmp."Ville"  \
	and tb."Date" = tmp."Date"' 

	ins_meteo = 'insert into meteo \
	select * \
	from meteo_tmp tmp  \
	where not exists (select 1 from meteo tb \
					where tb."Ville" = tmp."Ville" \
					and tb."Date" = tmp."Date")'

In [30]:
if rds_mode:
    # Connexion à la BDD
    conn = engine.connect()
    insp = inspect(engine)

    #Si on est en mode full (pas uniquement la mise à jour des prix) on créer la table à partir du DF 
    if (not insp.has_table('booking_hotel')) and mode == 'full':
        df_table_hotel.to_sql('booking_hotel',conn)
        df_table_price_hotel.to_sql('booking_hotel_price',conn)
        df_result.to_sql('meteo',conn)
    elif insp.has_table('booking_hotel') and mode == 'full':
        #Suppression des tables existantes
        stmt = text("DROP TABLE booking_hotel")
        result = conn.execute(stmt)
        stmt = text("DROP TABLE booking_hotel_price")
        result = conn.execute(stmt)
        stmt = text("DROP TABLE meteo")
        result = conn.execute(stmt)
        #Création à partir des nouvelles data récupérées
        df_table_hotel.to_sql('booking_hotel',conn)
        df_table_price_hotel.to_sql('booking_hotel_price',conn)
        df_result.to_sql('meteo',conn)
    #Cas ou on ne fait que de la mise à jour, on considère que les données des hotels n'ont pas évolué on met à jour la table price et la table météo
    elif insp.has_table('booking_hotel') and mode != 'full':
        #Création de table temporaire avec les données du moments
        df_table_price_hotel.to_sql('booking_hotel_price_tmp',conn)
        df_result.to_sql('meteo_tmp',conn)
        #mise à jour des Data (update insert)
        stmt = text(updt_hotel)
        result = conn.execute(stmt)
        stmt = text(updt_meteo)
        result = conn.execute(stmt)
        stmt = text(ins_hotel)
        result = conn.execute(stmt)
        stmt = text(ins_meteo)
        result = conn.execute(stmt)
        stmt = text("DROP TABLE meteo_tmp")
        result = conn.execute(stmt)
        stmt = text("DROP TABLE booking_hotel_price_tmp")
        result = conn.execute(stmt)

    conn.commit()

2023-05-05 08:00:40,497 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-05-05 08:00:40,498 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-05 08:00:40,539 INFO sqlalchemy.engine.Engine select current_schema()
2023-05-05 08:00:40,540 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-05 08:00:40,578 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-05-05 08:00:40,579 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-05 08:00:40,772 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-05 08:00:40,773 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

## Etape 6 : Lecture des données sur Amazon RDS

In [31]:
if rds_mode:
    #Récupération des dataframes depuis RDS
    rq = "SELECT \
    hti.ville ville, \
    hti.nom nom, \
    hti.url url, \
    hti.distance_center distance_center, \
    hti.latitude latitude, \
    hti.longitude longitude, \
    hti.description description, \
    htip.prix prix, \
    htip.note note \
    from booking_hotel hti \
    inner join booking_hotel_price htip on htip.ville = hti.ville \
    and htip.url = hti.url \
    and htip.nom = hti.nom"

    rq2 = 'SELECT \
    MAX("Ville") "Ville", \
    MAX("Latitude_return") "Latitude_return", \
    MAX("Longitude_return") "Longitude_return", \
    AVG("humidity") "avg_humidity", \
    AVG("wind_speed") "avg_wind_speed", \
    AVG("clouds") "avg_clouds", \
    AVG("pop") "avg_pop", \
    SUM("rain") "sum_rain", \
    AVG("feels_like.day") "avg_feel_temp" \
    FROM meteo  \
    where TO_TIMESTAMP("Date", \'YYYY-MM-DD\') BETWEEN TO_TIMESTAMP(\''+date_sejour+'\', \'YYYY-MM-DD\') and TO_TIMESTAMP(\''+date_fin_sejour+'\', \'YYYY-MM-DD\') \
    GROUP BY "Ville"'

    df_rds_hotel = pd.read_sql(rq,conn)
    df_rds_meteo = pd.read_sql(rq2,conn)
    df_rds_meteo = df_rds_meteo.fillna(0)
    #Sauvegarde des fichiers
    df_rds_hotel.to_csv('./testing_data/df_rds_hotel.csv')
    df_rds_meteo.to_csv('./testing_data/df_rds_meteo.csv')

else:
    #Lecture des fichiers CSV de sauvegarde
    df_rds_hotel = pd.read_csv('./testing_data/df_rds_hotel.csv')
    df_rds_meteo = pd.read_csv('./testing_data/df_rds_meteo.csv')



2023-05-05 08:00:50,719 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-05 08:00:50,719 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2023-05-05 08:00:50,720 INFO sqlalchemy.engine.Engine [cached since 9.948s ago] {'table_name': 'SELECT hti.ville ville, hti.nom nom, hti.url url, hti.distance_center distance_center, hti.latitude latitude, hti.longitude longitude, hti.descriptio ... (27 characters truncated) ... ix, htip.note note from booking_hotel hti inner join booking_hotel_price htip on htip.ville = hti.ville and htip.url = hti.url and htip.nom = hti.nom', 'param_1': 

In [32]:
#Création d'une colonne avec note réagencée pour le graphique (on utilise l'exponentiel pour accentuer les écarts de notes)
df_rds_hotel['note_aff'] = df_rds_hotel['note'].apply(lambda x : int(np.exp(x)/100) if not np.isnan(x) else np.exp(0.5))

In [34]:
#Création d'une colonne avec une description formatée pour le graphique 
def cut(chaine,taille):
    nb_mots = 0
    chaine_finale  = ''
    if chaine.find('Vous pouvez bénéficier d\'une réduction Gen') != -1 and chaine.find('pour économiser.')!=-1:
       chaine = chaine[chaine.find('pour économiser.')+16:]
    for i in chaine.split(' '):
        nb_mots += 1
        chaine_finale += i+' '
        if nb_mots == taille:
         chaine_finale += '</br> '
         nb_mots = 0
    return chaine_finale

df_rds_hotel['description_aff'] = df_rds_hotel['description'].apply(lambda x : cut(x,10) if x != None else x)

In [41]:
#Récupération des 20 Hotels les moins chers par ville 
df_rds_hotel_20  = df_rds_hotel
df_rds_hotel_20['rankbycity'] = df_rds_hotel_20.groupby("ville")["prix"].rank(method="dense", ascending=True)
mask = df_rds_hotel_20['rankbycity']< 21
df_rds_hotel_20 = df_rds_hotel_20[mask].sort_values(by=['ville','rankbycity'])

In [42]:
#Récupération des 5 destinations avec la meilleurs météo 
df_rds_meteo_5  = df_rds_meteo
#on créé un rank par mesure importante (temperature et pluie) avec des coefficients d'importance ici la pluie est 3 fois plus importante que la temperature
df_rds_meteo_5['rankbytemp'] = 2.5*df_rds_meteo_5["avg_feel_temp"].rank(method="dense", ascending=False)
df_rds_meteo_5['rankbyrain'] = 7.5*df_rds_meteo_5["sum_rain"].rank(method="dense", ascending=True)
#on fait la moyenne de ces notes
df_rds_meteo_5['notemoyenne'] = df_rds_meteo_5[['rankbytemp','rankbyrain']].mean(axis=1)
#On prend les 5 premières villes 
df_rds_meteo_5 = df_rds_meteo_5.sort_values(by='notemoyenne').iloc[0:5,:]


In [46]:
df_rds_hotel_20

,ville,nom,url,distance_center,latitude,longitude,description,prix,note,note_aff,description_aff,rankbycity
2195,Aigues Mortes,Noemys Aigues-Mortes - ex Mona Lisa Royal Hôtel,https://www.booking.com/hotel/fr/le-royal-hote...,1.2,43.576396,4.197818,Certaines chambres offrent une vue sur la pisc...,399.0,7.0,10.000000,Certaines chambres offrent une vue sur la pisc...,1.0
2202,Aigues Mortes,Résidence Odalys Fleur de Sel,https://www.booking.com/hotel/fr/residence-oda...,1.2,43.574094,4.179246,Vous pouvez bénéficier d'une réduction Genius ...,459.0,8.3,40.000000,"Répartis en petits duplex, tous les appartemen...",2.0
2193,Aigues Mortes,La Suite du Mas Lauseta - clim et park,https://www.booking.com/hotel/fr/la-maison-d-a...,0.6,43.567424,4.198007,L'appartement comprend une télévision à écran ...,460.0,NaN,1.648721,L'appartement comprend une télévision à écran ...,3.0
2207,Aigues Mortes,La maison sur la place,https://www.booking.com/hotel/fr/la-maison-sur...,250.0,43.565873,4.192959,Cette maison de vacances climatisée comprend 2...,473.0,8.5,49.000000,Cette maison de vacances climatisée comprend 2...,4.0
2206,Aigues Mortes,Hotel Canal Aigues Mortes,https://www.booking.com/hotel/fr/canal-aigues-...,0.8,43.572988,4.194825,Vous pouvez bénéficier d'une réduction Genius ...,491.0,8.5,49.000000,Toutes les chambres de cet hôtel 3 étoiles son...,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1433,Uzes,Le Mas des Alexandrins,https://www.booking.com/hotel/fr/le-mas-des-al...,1.9,43.995892,4.427467,Leur salle de bains privative est pourvue d'un...,745.0,9.4,120.000000,Leur salle de bains privative est pourvue d'un...,16.0
1421,Uzes,Uzes duplex atypique calme,https://www.booking.com/hotel/fr/uzes-duplex-a...,10.0,44.012052,4.419612,Vous pouvez bénéficier d'une réduction Genius ...,788.0,7.8,24.000000,"Cet appartement climatisé comprend 2 chambres,...",17.0
1430,Uzes,Le Sabotier,https://www.booking.com/hotel/fr/le-sabotier.f...,150.0,44.011359,4.418263,Vous pouvez bénéficier d'une réduction Genius ...,797.0,8.9,73.000000,Il comprend également une télévision par câble...,18.0
1429,Uzes,Villa Maélie,https://www.booking.com/hotel/fr/villa-maelie....,4.5,44.051169,4.403877,"Cette maison de vacances comprend 3 chambres, ...",894.0,8.9,73.000000,"Cette maison de vacances comprend 3 chambres, ...",19.0


In [45]:
df_rds_meteo_5

,Ville,Latitude_return,Longitude_return,avg_humidity,avg_wind_speed,avg_clouds,avg_pop,sum_rain,avg_feel_temp,rankbytemp,rankbyrain,notemoyenne
10,Bormes les Mimosas,43.1507,6.3419,57.0,5.096,57.8,0.176,0.59,20.544,22.5,7.5,15.00
1,Aix en Provence,43.5298,5.4475,38.0,5.936,49.8,0.254,2.32,23.070,10.0,30.0,20.00
24,Marseille,43.2962,5.3700,53.4,8.092,48.2,0.246,1.57,20.580,20.0,22.5,21.25
12,Cassis,43.2140,5.5396,54.8,7.638,49.6,0.182,1.01,20.284,27.5,15.0,21.25
27,Nimes,43.8374,4.3601,39.0,6.058,51.6,0.584,5.97,24.548,2.5,45.0,23.75


In [49]:
#filtre du Dataframe Hotel sur les 5 villes les plus chaudes
mask = df_rds_hotel_20['ville'].isin(df_rds_meteo_5['Ville'].to_list())
df_rds_hotel_20_in_5 = df_rds_hotel_20[mask]

## Etape 7 : Traçage des graphiques

In [43]:
#Traçage du graphique météo

fig = go.Figure()

#Définition d'une échelle de couleur pour les températures
echelle_couleur=[[0.0, "rgb(49,54,149)"],
                [0.1111111111111111, "rgb(69,117,180)"],
                [0.2222222222222222, "rgb(116,173,209)"],
                [0.3333333333333333, "rgb(171,217,233)"],
                [0.4444444444444444, "rgb(224,243,248)"],
                [0.5555555555555556, "rgb(254,224,144)"],
                [0.6666666666666666, "rgb(253,174,97)"],
                [0.7777777777777778, "rgb(244,109,67)"],
                [0.8888888888888888, "rgb(215,48,39)"],
                [1.0,  "rgb(165,0,38)"]]


fig.add_trace(
    go.Scattermapbox(
        lon = df_rds_meteo_5['Longitude_return'],
        lat = df_rds_meteo_5['Latitude_return'],
        customdata=df_rds_meteo_5[['avg_humidity','avg_wind_speed','avg_clouds','avg_pop','sum_rain','avg_feel_temp']].values.tolist(),
        marker = dict(
            size = df_rds_meteo_5['avg_feel_temp']*100,
            sizemode = 'area',
            color=df_rds_meteo_5['avg_feel_temp'],
            colorbar=dict(
                    title="Temperature"
                    ),
            colorscale=echelle_couleur
        ),
        name = 'Ville',
        text = df_rds_meteo_5['Ville'],
        #hoverinfo='none'
        hovertemplate="<b>Ville :</b> %{text} <br> \
<b>Température ressentie en journée :</b> %{customdata[5]} <br> \
<b>% Humidité :</b> %{customdata[0]} <br> \
<b>Vitesse du vent :</b> %{customdata[1]} <br> \
<b>% de nuages :</b> %{customdata[2]} <br> \
<b>Probabilité de pluie :</b> %{customdata[3]} <br> \
<b>Précipitation(mm) :</b> %{customdata[4]} <br>"
   )
)

fig.update_layout(
        width=1000,
        height=1000,
        mapbox=dict(
            accesstoken=mapbox_access_token,
            bearing=0,
            center=go.layout.mapbox.Center(
                lat=47,
                lon=2
        ),
        zoom=5.1,
        pitch=0
))

fig.show()

In [44]:
#Graphique pour les hotels 
#TODO #https://plotly.com/python/v3/cars-exploration/

fig = go.Figure()

echelle_couleur=[[0.0, "rgb(49,54,149)"],
                [0.1111111111111111, "rgb(69,117,180)"],
                [0.2222222222222222, "rgb(116,173,209)"],
                [0.3333333333333333, "rgb(171,217,233)"],
                [0.4444444444444444, "rgb(224,243,248)"],
                [0.5555555555555556, "rgb(254,224,144)"],
                [0.6666666666666666, "rgb(253,174,97)"],
                [0.7777777777777778, "rgb(244,109,67)"],
                [0.8888888888888888, "rgb(215,48,39)"],
                [1.0,  "rgb(165,0,38)"]]


fig.add_trace(
    go.Scattermapbox(
        lon = df_rds_hotel_20['longitude'],
        lat = df_rds_hotel_20['latitude'],
        customdata=df_rds_hotel_20[['note','distance_center','prix','url','nom','description_aff']].values.tolist(),
        marker = dict(
            size = df_rds_hotel_20['note_aff'],
            sizemode = 'area',
            color=df_rds_hotel_20['prix'],
            colorbar=dict(
                    title="Tarif(€)"
                    )
        ),
        name = 'Nom',
        text = df_rds_hotel_20['nom'],
        #hoverinfo='none'
        hovertemplate="<b>Hotel :</b> %{text} <br> \
<b>Note :</b> %{customdata[0]} <br> \
<b>Prix :</b> %{customdata[2]} <br> \
<b>Distance du centre :</b> %{customdata[1]} <br> \
<b>Lien :</b> %{customdata[3]} <br> \
<b>escription :</b> %{customdata[5]} <br> "
   )
)

fig.update_layout(
        width=1000,
        height=1000,
        mapbox=dict(
            accesstoken=mapbox_access_token,
            bearing=0,
            center=go.layout.mapbox.Center(
                lat=47,
                lon=2
        ),
        zoom=5.1,
        pitch=0
))

fig.show()

In [51]:
#Graphique pour les hotels dans nos 5 villes
#TODO #https://plotly.com/python/v3/cars-exploration/

fig = go.Figure()

echelle_couleur=[[0.0, "rgb(49,54,149)"],
                [0.1111111111111111, "rgb(69,117,180)"],
                [0.2222222222222222, "rgb(116,173,209)"],
                [0.3333333333333333, "rgb(171,217,233)"],
                [0.4444444444444444, "rgb(224,243,248)"],
                [0.5555555555555556, "rgb(254,224,144)"],
                [0.6666666666666666, "rgb(253,174,97)"],
                [0.7777777777777778, "rgb(244,109,67)"],
                [0.8888888888888888, "rgb(215,48,39)"],
                [1.0,  "rgb(165,0,38)"]]


fig.add_trace(
    go.Scattermapbox(
        lon = df_rds_hotel_20_in_5['longitude'],
        lat = df_rds_hotel_20_in_5['latitude'],
        customdata=df_rds_hotel_20_in_5[['note','distance_center','prix','url','nom','description_aff']].values.tolist(),
        marker = dict(
            size = df_rds_hotel_20_in_5['note_aff'],
            sizemode = 'area',
            color=df_rds_hotel_20_in_5['prix'],
            colorbar=dict(
                    title="Tarif(€)"
                    )
        ),
        name = 'Nom',
        text = df_rds_hotel_20_in_5['nom'],
        #hoverinfo='none'
        hovertemplate="<b>Hotel :</b> %{text} <br> \
<b>Note :</b> %{customdata[0]} <br> \
<b>Prix :</b> %{customdata[2]} <br> \
<b>Distance du centre :</b> %{customdata[1]} <br> \
<b>Lien :</b> %{customdata[3]} <br> \
<b>escription :</b> %{customdata[5]} <br> "
   )
)

fig.update_layout(
        width=1000,
        height=1000,
        mapbox=dict(
            accesstoken=mapbox_access_token,
            bearing=0,
            center=go.layout.mapbox.Center(
                lat=47,
                lon=2
        ),
        zoom=5.1,
        pitch=0
))

fig.show()

## Etape 8 : Ouverture...

In [7]:
#Pour les graphiques : Proposer la gestion des filtres en live sur les graphes
#Suite à l'apprentissage de streamlit : Utiliser Streamlit !! 
#cf #https://plotly.com/python/v3/cars-exploration/

#Pour les appels API : 
#Passer les scripts en Asynchrone de sorte à gagner du temps 

#Pour le scrapping : 
#Comme nous avons une liste finie de ville à scrapper, l'idéal serait de lancer 1 ville / process 
#Mettre en place des proxy rotatif pour éviter le ban
#Récupérer des photos et afficher ca sur une map html comme un site

#Pour la partie ETL : 
#Ameliorer le système de mise à jour avec une date d'insertion des données et définir une période de validité de ces données

#Pour la sélection des hotels et de la méteo 
#Demander à l'utilisateur de choisir ses préférences entre le prix etc... et filtrer tous les hotels de toutes les villes sur ces critères
    #Sélection des 20 hotels à afficher
    #On prend les hotels avec note générale de 8
    #Avec une distance du centre de moins de 1km
    #avec un prix compris entre moyenne - 1 ecart type et moyenne + 1 ecart type
    #mask = (df_rds_hotel['note'] > 8) & (df_rds_hotel['distance_center'] < 1) & (df_rds_hotel['prix'] < (df_rds_hotel['prix'].mean() + df_rds_hotel['prix'].mean())) & (df_rds_hotel['prix'] > (df_rds_hotel['prix'].mean() - df_rds_hotel['prix'].mean()))
#Matcher avec les meilleurs météo 
#Récupérer une liste de 3 Hotels qui sont parfaits